# Mission 2-1

1. 파일 이름에서 정보 추출 함수 (extract_info_from_filename)
def extract_info_from_filename(filename, type):
    parts = filename.split('_')
    info = {
        'W_T': parts[0],
        'image_id': parts[1],
        'era': parts[2],
        'style': parts[3],
        'gender': parts[4] if type==0 else parts[4].split('.')[0],
        **({'survey_id': parts[5].split('.')[0]} if type == 0 else {})
    }
    return info

설명:
- 이 함수는 주어진 파일 이름을 split하여 필요한 정보를 딕셔너리 형태로 반환합니다.
- 파라미터: type 파라미터는 파일 유형을 나타내며, type=0일 때는 라벨 파일(.json), type=1일 때는 이미지 파일(.jpg)로 처리합니다.
- type=0인 경우, survey_id도 포함하여 라벨 파일에서 추가적으로 필요한 정보를 딕셔너리에 저장합니다.

Mission 2-1 요구 사항:
- 이 함수는 파일명 형식을 활용해 각 이미지의 image_id, 성별(gender), 스타일(style) 등의 메타 데이터를 추출하여 라벨 데이터와 매칭할 준비를 합니다.

2. 폴더 내 파일 처리 함수 (process_folder)
def process_folder(folder_path):
    data = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            info = extract_info_from_filename(filename, 0)
            info['filename'] = filename
            data.append(info)
        
        elif filename.endswith('.jpg'):
            info = extract_info_from_filename(filename, 1)
            info['filename'] = filename
            data.append(info)
            
    return data

설명:
- process_folder 함수는 지정된 폴더의 모든 파일을 순회하며, 파일이 .json 확장자인 경우 라벨 파일로, .jpg 확장자인 경우 이미지 파일로 처리합니다.
- extract_info_from_filename 함수를 호출하여 파일명을 통해 필요한 정보를 추출하고, data 리스트에 저장합니다.

Mission 2-1 요구 사항:
- 학습 및 검증 데이터 폴더의 라벨 및 이미지 파일을 각각 처리하여 필요한 정보를 추출하고, 이후 데이터 필터링에 사용할 수 있도록 준비합니다.

3. 공통 항목으로 라벨 데이터 필터링 (filter_labels_with_images)
def filter_labels_with_images(label_df, image_df, common_columns):
    return label_df[
        label_df[common_columns].apply(tuple, axis=1).isin(
            image_df[common_columns].apply(tuple, axis=1))
    ]

설명:
- 이 함수는 이미지 데이터(image_df)에 포함된 항목과 일치하는 라벨 데이터만 필터링하여 반환합니다.
- 작동 원리: label_df와 image_df에서 각각 common_columns 열을 이용해 각 행을 튜플 형태로 변환하고, isin()을 사용하여 이미지와 일치하는 라벨만 필터링합니다.

Mission 2-1 요구 사항:
- 문제 조건에 따라 유효한 라벨만을 남기기 위해, 이미지와 일치하는 라벨 데이터만 필터링하여 남겨두는 작업을 수행합니다.

4. 데이터 처리
train_image_data = process_folder('data/training_image')
valid_image_data = process_folder('data/validation_image')
train_label_data = process_folder('data/training_label')
valid_label_data = process_folder('data/validation_label')

설명:
- 학습과 검증 이미지 및 라벨 데이터의 경로를 지정하고, 각각 process_folder 함수를 호출하여 라벨과 이미지 데이터 정보를 모두 추출합니다.
- 결과는 train_image_data, valid_image_data, train_label_data, valid_label_data로 저장됩니다.

Mission 2-1 요구 사항:
- 각 데이터셋을 불러와 필터링 작업을 수행할 준비를 합니다.

5. 데이터를 DataFrame으로 변환
train_label_df = pd.DataFrame(train_label_data)
valid_label_df = pd.DataFrame(valid_label_data)
train_image_df = pd.DataFrame(train_image_data)
valid_image_df = pd.DataFrame(valid_image_data)

설명:
- 추출된 데이터(train_label_data, valid_label_data, train_image_data, valid_image_data)를 각각 DataFrame으로 변환합니다.
- 이는 이후 데이터 분석 및 필터링을 보다 쉽게 수행할 수 있도록 하기 위함입니다.

Mission 2-1 요구 사항:
- DataFrame 형식으로 변환하여 필터링 및 통계 처리가 용이하게 만듭니다.

6. 필터링 전 데이터 개수 확인
print("Training 라벨 데이터 개수(필터링 전): ", train_label_df.shape[0])
print("Validation 라벨 데이터 개수(필터링 전): ", valid_label_df.shape[0])

설명:
- 필터링을 수행하기 전에 학습 및 검증 라벨 데이터의 개수를 출력하여, 필터링 후의 데이터 개수와 비교할 수 있도록 합니다.

Mission 2-1 요구 사항:
- 필터링 전과 후의 데이터 수를 비교하여, 유효한 라벨 데이터만 남았는지 확인할 수 있습니다.

7. 공통된 열을 기준으로 데이터셋에서 매칭되는 라벨 필터링
common_columns = ['W_T', 'image_id', 'era', 'style', 'gender']
filtered_train_label_df = filter_labels_with_images(train_label_df, train_image_df, common_columns)
filtered_valid_label_df = filter_labels_with_images(valid_label_df, valid_image_df, common_columns)

설명:
- common_columns를 기준으로 학습 및 검증 라벨 데이터에서 이미지와 매칭되는 라벨만 필터링합니다.
- 필터링 결과는 filtered_train_label_df와 filtered_valid_label_df에 저장됩니다.

Mission 2-1 요구 사항:
- 유효한 라벨 데이터만 남기는 작업으로, 이미지 데이터와 매칭되는 라벨만을 남겨 Mission 2-1의 요구 사항을 만족합니다.

8. 필터링 후 데이터 개수 출력
print("\nTraining 라벨 데이터 개수(필터링 후): ", filtered_train_label_df.shape[0])
print("Validation 라벨 데이터 개수(필터링 후): ", filtered_valid_label_df.shape[0])

설명:
- 필터링 후 데이터의 개수를 출력하여 필터링 전과 비교할 수 있도록 합니다.
이를 통해 필터링 작업이 올바르게 수행되었는지 검증할 수 있습니다.

9. CSV 파일로 저장
# filtered_train_label_df.to_csv('data/filtered_train_label_metadata.csv', index=False)
# filtered_valid_label_df.to_csv('data/filtered_valid_label_metadata.csv', index=False)

설명:
- 필터링된 학습 및 검증 라벨 데이터를 CSV 파일로 저장하여, 이후 분석 및 모델 학습에 사용될 수 있도록 준비합니다.
- 이 과정은 주석 처리되어 있지만, 필요 시 주석을 제거해 CSV 파일로 저장할 수 있습니다.

Mission 2-1 요구 사항:
최종적으로 유효한 라벨 데이터셋을 별도 파일로 저장함으로써 데이터 준비를 완료하고, 이후의 작업에 사용할 수 있습니다.



1. 성별 & 스타일별 개수를 세는 함수 (count_by_gender_and_style)
def count_by_gender_and_style(df):
    return df.groupby(['gender', 'style']).size().reset_index(name='count')

설명:
- 이 함수는 주어진 데이터프레임 df에서 gender와 style 열을 기준으로 그룹화하여 각 성별과 스타일 조합의 개수를 세고, 결과를 DataFrame 형태로 반환합니다.
- groupby를 통해 성별과 스타일을 그룹화한 후 size()를 사용하여 각 그룹의 개수를 세고, reset_index(name='count')로 각 조합과 개수를 포함한 새로운 데이터프레임을 생성합니다.

Mission 2-1 요구 사항:
- 이 함수는 유효한 라벨 데이터를 기반으로 성별과 스타일별로 통계를 집계해야 한다는 Mission 2-1의 요구 사항을 충족합니다.

2. 유효한 라벨 데이터에서 성별 & 스타일별 개수 집계
train_gender_style_counts = count_by_gender_and_style(filtered_train_label_df)
valid_gender_style_counts = count_by_gender_and_style(filtered_valid_label_df)

설명:
- Mission 2-1에서 필터링된 학습 및 검증 라벨 데이터프레임(filtered_train_label_df, filtered_valid_label_df)을 입력으로 하여 count_by_gender_and_style 함수를 호출합니다.
- 학습 데이터(train_gender_style_counts)와 검증 데이터(valid_gender_style_counts)에서 성별 및 스타일 조합별 개수를 각각 집계합니다.

Mission 2-1 요구 사항:
- 이 작업을 통해 유효한 학습 및 검증 데이터에서 각 성별과 스타일 조합에 해당하는 이미지 개수를 집계하게 되므로, Mission 2-1의 요구 사항에 부합합니다.


3. 결과 출력
print("Training gender & style counts:")
print(train_gender_style_counts)

print("\nValidation gender & style counts:")
print(valid_gender_style_counts)

설명:
- 학습 및 검증 데이터에 대한 성별 및 스타일별 통계 결과를 출력하여 개별 성별 및 스타일 조합에 속한 이미지 수를 확인할 수 있습니다.

Mission 2-1 요구 사항:
- 통계 결과를 확인할 수 있도록 출력하여, 데이터의 전반적인 분포를 파악할 수 있게 합니다.

4. 결과 제출용 CSV 파일 저장
train_gender_style_counts.to_csv('Mission2-1 Training Result.csv', index=False)
valid_gender_style_counts.to_csv('Mission2-1 Validation Result.csv', index=False)

설명:
- to_csv 메서드를 사용해 학습 및 검증 데이터의 성별 & 스타일별 통계 결과를 각각 CSV 파일로 저장합니다.
- index=False는 데이터프레임의 인덱스가 CSV 파일에 포함되지 않도록 설정하여, 간결한 형태로 결과를 저장합니다.

Mission 2-1 요구 사항:
- CSV 파일 형식으로 통계를 저장하여 Mission 2-1의 결과를 제출할 수 있도록 준비합니다. 학습 결과는 Mission2-1 Training Result.csv에, 검증 결과는 Mission2-1 Validation Result.csv에 각각 저장됩니다.


In [ ]:
import os
import pandas as pd

# 파일 이름에서 정보 추출 (type = {label= 0, image= 1})
def extract_info_from_filename(filename, type):
    parts = filename.split('_')
    info = {
        'W_T': parts[0],
        'image_id': parts[1],
        'era': parts[2],
        'style': parts[3],
        'gender': parts[4] if type==0 else parts[4].split('.')[0],
        **({'survey_id': parts[5].split('.')[0]} if type == 0 else {})
    }
    return info

# 폴더에 있는 파일 처리
def process_folder(folder_path):
    data = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            info = extract_info_from_filename(filename, 0)
            info['filename'] = filename
            data.append(info)

        elif filename.endswith('.jpg'):
            info = extract_info_from_filename(filename, 1)
            info['filename'] = filename
            data.append(info)

    return data

# 공통된 항목을 비교하여 매칭되는 라벨만 필터링하는 함수
def filter_labels_with_images(label_df, image_df, common_columns):
    return label_df[
        label_df[common_columns].apply(tuple, axis=1).isin(
            image_df[common_columns].apply(tuple, axis=1))
    ]


# 데이터 처리
train_image_data = process_folder('data/training_image')
valid_image_data = process_folder('data/validation_image')
train_label_data = process_folder('data/training_label')
valid_label_data = process_folder('data/validation_label')

# 데이터를 DataFrame으로 변환
train_label_df = pd.DataFrame(train_label_data)
valid_label_df = pd.DataFrame(valid_label_data)
train_image_df = pd.DataFrame(train_image_data)
valid_image_df = pd.DataFrame(valid_image_data)

# 필터링 전 데이터 개수
print("Training 라벨 데이터 개수(필터링 전): ", train_label_df.shape[0])
print("Validation 라벨 데이터 개수(필터링 전): ", valid_label_df.shape[0])

# 공통된 열 정의
common_columns = ['W_T', 'image_id', 'era', 'style', 'gender']

# 데이터셋에서 매칭되는 라벨만 필터링
filtered_train_label_df = filter_labels_with_images(train_label_df, train_image_df, common_columns)
filtered_valid_label_df = filter_labels_with_images(valid_label_df, valid_image_df, common_columns)

# 필터링된 결과 출력
print("\nTraining 라벨 데이터 개수(필터링 후): ", filtered_train_label_df.shape[0])
print("Validation 라벨 데이터 개수(필터링 후): ", filtered_valid_label_df.shape[0])

# CSV 파일로 저장
# filtered_train_label_df.to_csv('data/filtered_train_label_metadata.csv', index=False)
# filtered_valid_label_df.to_csv('data/filtered_valid_label_metadata.csv', index=False)

Training 라벨 데이터 개수(필터링 전):  211346
Validation 라벨 데이터 개수(필터링 전):  36383

Training 라벨 데이터 개수(필터링 후):  16096
Validation 라벨 데이터 개수(필터링 후):  4105


In [ ]:
# Mission 2-1
def count_by_gender_and_style(df):
    return df.groupby(['gender', 'style']).size().reset_index(name='count')


# 데이터셋에서 성별 & 스타일 별 개수 세기
train_gender_style_counts = count_by_gender_and_style(filtered_train_label_df)
valid_gender_style_counts = count_by_gender_and_style(filtered_valid_label_df)

# 결과 출력
print("Training gender & style counts:")
print(train_gender_style_counts)

print("\nValidation gender & style counts:")
print(valid_gender_style_counts)

# 결과 제출용
train_gender_style_counts.to_csv('Mission2-1 Training Result.csv', index=False)
valid_gender_style_counts.to_csv('Mission2-1 Validation Result.csv', index=False)

Training gender & style counts:
   gender           style  count
0       M            bold   1007
1       M          hiphop   1070
2       M          hippie   1502
3       M             ivy   1608
4       M     metrosexual   1045
5       M            mods   1458
6       M        normcore    644
7       M  sportivecasual    845
8       W      athleisure    378
9       W   bodyconscious    455
10      W        cityglam    231
11      W         classic    360
12      W           disco    130
13      W         ecology    223
14      W        feminine    719
15      W      genderless    146
16      W          grunge     90
17      W          hiphop    138
18      W          hippie    308
19      W          kitsch    261
20      W        lingerie    156
21      W          lounge     82
22      W        military    109
23      W         minimal    643
24      W        normcore    278
25      W        oriental    282
26      W          popart    193
27      W       powersuit    531
28      W  

# Mission 2-2

1. 설문 조사에서 필요한 정보만 추출하는 함수 (extract_info_from_survey)

import json

def extract_info_from_survey(filenames, type):
    data = []
    dir = "data/training_label/" if type==0 else "data/validation_label/"
    
    for filename in filenames:
        with open(dir + filename, 'r') as file:
            survey_data = json.load(file)
            extracted_data = {
                'R_id': survey_data['user']['R_id'],
                'image_id': survey_data['imgName'],
                'Q5': survey_data['item']['survey']['Q5']
            }
            data.append(extracted_data)
            
    return data

설명:
- 설문 조사 데이터에서 필요한 정보(R_id, image_id, Q5)만 추출하여 리스트 data에 저장합니다.
- R_id는 응답자 ID, image_id는 이미지 식별자, Q5는 스타일 선호 여부를 나타냅니다.
- 파라미터:
filenames는 필터링된 라벨 데이터프레임에서 가져온 파일 이름 목록입니다.
type은 0(학습 데이터) 또는 1(검증 데이터)으로 구분하여 올바른 디렉토리 경로를 선택합니다.

Mission 2-2 요구 사항:
- 설문 조사 파일에서 각 응답자의 ID와 이미지별 선호 여부 정보를 추출하여 이후 응답자별로 선호/비선호 스타일 정보를 구성할 수 있도록 준비합니다.

2. 데이터 추출 및 DataFrame으로 변환
train_survey_data = extract_info_from_survey(filtered_train_label_df['filename'], 0)
valid_survey_data = extract_info_from_survey(filtered_valid_label_df['filename'], 1)

train_survey_df = pd.DataFrame(train_survey_data)
valid_survey_df = pd.DataFrame(valid_survey_data)

설명:
- 학습용 라벨 데이터와 검증용 라벨 데이터에서 필요한 정보를 extract_info_from_survey 함수를 사용해 각각 추출하고, 이를 DataFrame으로 변환합니다.
- train_survey_df와 valid_survey_df는 응답자 ID(R_id), 이미지 ID(image_id), 스타일 선호도(Q5)를 포함한 테이블 형태의 데이터입니다.

Mission 2-2 요구 사항:
- 학습과 검증 설문 데이터를 각각 DataFrame으로 변환하여, 응답자별 선호/비선호 데이터를 쉽게 분석할 수 있도록 준비합니다.

3. 응답자 ID별 설문조사 횟수 집계 및 상위 100명 추출
concat_survey_df = pd.concat([train_survey_df, valid_survey_df], ignore_index=True)
R_id_count = concat_survey_df['R_id'].value_counts().head(100)
users = R_id_count.index.tolist()

설명:
- 학습과 검증 데이터프레임을 pd.concat으로 병합하여 응답자별 설문 횟수를 집계하고, 설문 횟수가 많은 상위 100명의 응답자 ID를 추출합니다.
- value_counts()로 응답자별 설문 횟수를 세고, head(100)으로 상위 100명의 응답자 ID를 리스트(users)에 저장합니다.

Mission 2-2 요구 사항:
응답자 중 설문조사에 가장 많이 참여한 상위 100명을 추출하여 이후의 선호/비선호 스타일 정보를 정리할 수 있도록 합니다.

4. 응답자가 응답한 설문 데이터를 필터링하고 선호도 정보 추출
data = []
for user in users:
    user_related_train = train_survey_df[train_survey_df['R_id'].isin([user])]
    user_related_valid = valid_survey_df[valid_survey_df['R_id'].isin([user])]
    info = {
        'R_id': user,
        'training': {
            'like': user_related_train.loc[user_related_train['Q5'] == 2, 'image_id'].tolist(),
            'dislike': user_related_train.loc[user_related_train['Q5'] == 1, 'image_id'].tolist()
        },
        'validation': {
            'like': user_related_valid.loc[user_related_valid['Q5'] == 2, 'image_id'].tolist(),
            'dislike': user_related_valid.loc[user_related_valid['Q5'] == 1, 'image_id'].tolist()
        }
    }
    data.append(info)

설명:
- 상위 100명 응답자 각각에 대해 선호 및 비선호 이미지 목록을 정리합니다.
user_related_train과 user_related_valid는 각 응답자가 참여한 학습 및 검증 데이터의 설문 정보입니다.
- Q5 == 2는 선호 이미지를 나타내며, Q5 == 1은 비선호 이미지를 나타냅니다.
like와 dislike 키에 이미지 ID를 리스트 형태로 저장하여 응답자별로 선호 및 비선호 이미지를 구분합니다.
- 최종적으로 응답자의 ID와 그에 따른 선호/비선호 이미지 정보를 data 리스트에 저장합니다.

Mission 2-2 요구 사항:
- 각 응답자의 설문조사 데이터를 분석하여 응답자의 선호 이미지와 비선호 이미지를 분리해 정리하는 작업으로, Mission 2-2의 요구 사항을 충족합니다.

5. 결과 저장
with open('Mission2-2 Result.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)

설명:
- data 리스트를 JSON 형식으로 변환하여 Mission2-2 Result.json 파일에 저장합니다.
- indent=4로 설정하여 JSON 파일의 가독성을 높였습니다.

Mission 2-2 요구 사항:
- 최종 결과를 JSON 파일로 저장하여 제출할 수 있는 형식을 만듭니다. 이 파일에는 응답자별 선호 및 비선호 이미지 정보가 구조화되어 저장됩니다.

In [ ]:
# Mission 2-2
import json

# 설문조사에서 미션 수행에 필요한 정보만 추출 (type = {train= 0, valid= 1})
def extract_info_from_survey(filenames, type):
    data = []
    dir = "data/training_label/" if type==0 else "data/validation_label/"

    for filename in filenames:
        with open(dir+filename, 'r') as file:
            survey_data = json.load(file)
            extracted_data = {
                'R_id': survey_data['user']['R_id'],
                'image_id': survey_data['imgName'],
                'Q5': survey_data['item']['survey']['Q5']
            }
            data.append(extracted_data)

    return data


# 데이터 추출
train_survey_data = extract_info_from_survey(filtered_train_label_df['filename'], 0)
valid_survey_data = extract_info_from_survey(filtered_valid_label_df['filename'], 1)

# 추출된 데이터를 DataFrame으로 변환
train_survey_df = pd.DataFrame(train_survey_data)
valid_survey_df = pd.DataFrame(valid_survey_data)

# 설문조사 결과가 많은 응답자 TOP 100
concat_survey_df = pd.concat([train_survey_df, valid_survey_df], ignore_index=True)
R_id_count = concat_survey_df['R_id'].value_counts().head(100)
users = R_id_count.index.tolist()

# 응답자가 설문에 응답한 사진 데이터만 선택 후, 선호&비선호 추출
data = []
for user in users:
    user_related_train = train_survey_df[train_survey_df['R_id'].isin([user])]
    user_related_valid = valid_survey_df[valid_survey_df['R_id'].isin([user])]
    info = {
        'R_id': user,
        'training': {
            'like': user_related_train.loc[user_related_train['Q5']==2, 'image_id'].tolist(),
            'dislike': user_related_train.loc[user_related_train['Q5']==1, 'image_id'].tolist()
        },
        'validation': {
            'like': user_related_valid.loc[user_related_valid['Q5']==2, 'image_id'].tolist(),
            'dislike': user_related_valid.loc[user_related_valid['Q5']==1, 'image_id'].tolist()
        }
    }
    data.append(info)

# 결과 저장
with open('Mission2-2 Result.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)